In [ ]:
import pandas as pd
import json
import os
from pathlib import Path
from openai import OpenAI
from dotenv import load_dotenv


load_dotenv()  # Carrega variáveis do arquivo .env


if "OPENAI_API_KEY" not in os.environ:
    raise ValueError("Por favor, configure a variável de ambiente OPENAI_API_KEY")


try:
    flights = pd.read_csv(
        'data/processed/flights_with_predictions.csv',
        parse_dates=['departure_time', 'scheduled_arrival', 'actual_arrival']
    )
except FileNotFoundError:
    raise FileNotFoundError("Arquivo de dados não encontrado. Execute os notebooks anteriores primeiro.")


def prepare_summary(df):
    """Prepara um resumo serializável dos dados para análise pelo LLM"""
    
   
    summary = {
        "total_flights": int(len(df)),
        "problem_flights": int(df['is_problem'].sum()),
        "problem_rate": f"{(df['is_problem'].mean() * 100):.1f}%",
        "avg_delay": float(df[df['status'] == 'delayed']['delay_minutes'].mean()),
        "max_delay": float(df[df['status'] == 'delayed']['delay_minutes'].max())
    }
    
    
    cancellations = df[df['status'] == 'canceled']['cancellation_reason'].value_counts()
    summary["cancellation_reasons"] = cancellations.to_dict()
    
    
    anomalies = df[df['is_anomaly'] == 1]['status'].value_counts()
    summary["common_anomalies"] = anomalies.to_dict()
    
    
    problem_routes = (
        df[df['is_problem'] == 1]
        .groupby(['origin', 'destination'])
        .size()
        .nlargest(3)
    )
    summary["top_problem_routes"] = {
        f"{orig}-{dest}": int(count) 
        for (orig, dest), count in problem_routes.items()
    }
    
    
    problem_airlines = (
        df[df['is_problem'] == 1]['airline']
        .value_counts()
        .head(3)
    )
    summary["problem_by_airline"] = problem_airlines.to_dict()
    
    return summary


def generate_llm_report(data_summary):
    """Gera um relatório de análise usando a API da OpenAI"""
    
    client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
    
    prompt = """Analise este resumo de dados de voos e gere um relatório detalhado em português:

{data_summary}

O relatório deve conter:
1. Visão geral dos dados (total de voos, taxa de problemas)
2. Análise dos principais problemas (atrasos, cancelamentos)
3. Rotas mais problemáticas
4. Companhias aéreas com mais ocorrências
5. Recomendações para melhorias operacionais
6. Sugestões para coleta de dados futuros

Formate o relatório em markdown com títulos e bullet points.""".format(
    data_summary=json.dumps(data_summary, indent=2, ensure_ascii=False)
)
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Você é um analista de dados de aviação experiente."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=4096
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Erro na chamada da API: {str(e)}")
        raise


if __name__ == "__main__":
    try:
        
        Path('data/reports').mkdir(parents=True, exist_ok=True)
        
        
        summary = prepare_summary(flights)
        print("Sumário preparado com sucesso!")
        
        
        report = generate_llm_report(summary)
        
        
        report_path = 'data/reports/flight_analysis_report.md'
        with open(report_path, 'w', encoding='utf-8') as f:
            f.write(report)
        
        print(f"\nRelatório gerado com sucesso em: {report_path}")
        print("\nPré-visualização do relatório:")
        print(report[:500] + "...")
        
    except Exception as e:
        print(f"\nErro durante a execução: {str(e)}")

Sumário preparado com sucesso!

Relatório gerado com sucesso em: data/reports/flight_analysis_report.md

Pré-visualização do relatório:
# Relatório de Análise de Dados de Voos

## 1. Visão Geral dos Dados
- Total de voos: 5000
- Voos com problemas: 980
- Taxa de problemas: 19.6%
- Atraso médio: 221.08 minutos
- Maior atraso: 478 minutos

## 2. Análise dos Principais Problemas
- Motivos de cancelamento:
  - Tráfego aéreo: 64
  - Condições climáticas: 50
  - Tripulação: 45
  - Problemas mecânicos: 38
- Anomalias comuns:
  - Voos no horário: 661
  - Voos cancelados: 196
  - Voos atrasados: 75
  - Voos desviados: 43
  - Voos com exc...
